In [9]:
# Imports
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import datetime
from twilio.rest import Client

In [1]:
load_dotenv()

# GLOBAL API Setup
ALPHA_API_KEY = os.environ.get('ALPHA_API_KEY')
NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
TWILIO_ACCOUNT_SID = os.environ.get('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.environ.get('TWILIO_AUTH_TOKEN')
FROM_WPP = os.environ.get('FROM_WPP')
TO_WPP = os.environ.get('TO_WPP')
ALPHA_URL = os.environ.get('ALPHA_URL')
ALPHA_SYMBOL_SEARCH_URL = os.environ.get('ALPHA_SYMBOL_SEARCH_URL')
NEWS_URL = os.environ.get('NEWS_URL')


NameError: name 'load_dotenv' is not defined

In [13]:
def get_symbol(keyword: str) -> tuple[str, str]:
    url = ALPHA_SYMBOL_SEARCH_URL
    params = {
        'function': 'SYMBOL_SEARCH',
        'keywords': keyword,
        'apikey': ALPHA_API_KEY
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    print(f"Response: {response}")
    print(f"code: {response.status_code}")
    raw_data = response.json()
    print(raw_data)
    company_name = raw_data['bestMatches'][0]['2. name']
    stock_symbol = raw_data['bestMatches'][0]['1. symbol']
    
    return company_name, stock_symbol

In [ ]:
company_name, stock_symbol = get_symbol('Tesla')
print(f"Company Name: {company_name}")
print(f"Stock Symbol: {stock_symbol}")

In [4]:

# Stock variation calculation
def get_stock_variation(stock_symbol: str) -> bool:
    """Fetch stock variation percentage from the Alphavantage API and check if it exceeds 5%."""
    alpha_params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': stock_symbol.upper(),
        'apikey': ALPHA_API_KEY
    }
    
    response = requests.get(ALPHA_URL, params=alpha_params)
    response.raise_for_status()
    raw_data = response.json()
    data = raw_data["Time Series (Daily)"]
    dates = sorted(data.keys(), reverse=True) # Get sorted available dates from API
    
    try:
        yesterday_date = dates[0]
        two_days_ago_date = dates[1]
    except IndexError:
        print("Not enough data to calculate stock variation")
        return False
    
    # Fetch close prices
    yesterday_close = float(data[yesterday_date]["4. close"])
    two_days_close = float(data[two_days_ago_date]["4. close"])
    
    percentage_difference = round(((yesterday_close - two_days_close) / two_days_close * 100),2)
    print(f'Stock price variation:{percentage_difference}')
    
    if abs(percentage_difference) >= 1:
        print("The variation is more than 1%")
        return True
    else:
        print("The variation is within 1%")
        return False

In [5]:
# Fetch news articles
def get_news(query,lang) -> list:
    """Retrieve top 3 news articles based on the query."""
    
    yesterday_date = datetime.datetime.now() - datetime.timedelta(days=1)
    news_params = {
        'apiKey': os.environ.get('NEWS_API_KEY'),
        'qInTitle': query,
        'from': yesterday_date,
        'language': lang,
        'sortBy':'relevancy'
    }
    
    response = requests.get(NEWS_URL, news_params)
    response.raise_for_status()
    data = response.json() # o: python dict
    # print(data)
    # print(type(data))

    return data['articles'][0:3]

In [6]:
# Send wpp message
def send_wpp_message(message):
    """Send a WhatsApp message using Twilio."""
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    msn = client.messages.create( 
        body=message,
        from_=FROM_WPP,
        to=TO_WPP,
        )
    return msn.status

In [7]:
def main():
    company_name, stock_symbol = get_symbol('Tesla')
    language = 'en'
    print(f"Company Name: {company_name}")
    print(f"Stock Symbol: {stock_symbol}")
    stock_price_variation = get_stock_variation(stock_symbol)
    news = get_news(company_name, language)
    if stock_price_variation:
        print("Stock variation detected. Fetching news articles...")
        print(news)
        
        for article in news:
            message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
            send_wpp_message(message)
            print(f"Message sent: {message}")
    else:
        print("No stock variation detected")
        #print(news)
        #message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
        #set_wpp_message(message)
        #print(f"Message sent: {message}")
        
        

In [ ]:
main()